In [46]:
# DATASET_NUM = 0
DATASET_NUM = 1

import pandas as pd

pair_list = ["BTCUSDT","SOLUSDT","ETHUSDT"]
dataset = pd.read_parquet(f"data/dataset_{DATASET_NUM}.parquet")

btc = dataset[dataset.PAIR == "BTCUSDT"]
eth = dataset[dataset.PAIR == "ETHUSDT"]
sol = dataset[dataset.PAIR == "SOLUSDT"]

In [47]:
sol = sol.drop(columns=["PAIR","open","high","low"])
sol.columns

Index(['close', 'volume', 'qav', 'num_trades', 'taker_base_vol',
       'taker_quote_vol'],
      dtype='object')

In [48]:
sol["price_var"] = sol["close"].diff()
sol["next_price_var"] = sol["close"].diff().shift(-1)
lag = 5 

for i in range(1, lag):
    sol[f"price_var_lag_{i}"] = sol["price_var"].shift(i)

sol = sol.iloc[lag:-1]  # to drop nan generated by shifts

In [49]:
sol.tail(3)

,close,volume,qav,num_trades,taker_base_vol,taker_quote_vol,price_var,next_price_var,price_var_lag_1,price_var_lag_2,price_var_lag_3,price_var_lag_4
2025-04-16 23:59:57,132.92,0.000,0.00000,0.0,0.000,0.00000,0.00,0.00,-0.01,0.01,0.00,0.01
2025-04-16 23:59:58,132.92,0.000,0.00000,0.0,0.000,0.00000,0.00,0.05,0.00,-0.01,0.01,0.00
2025-04-16 23:59:59,132.97,112.299,14931.08382,143.0,112.299,14931.08382,0.05,0.08,0.00,0.00,-0.01,0.01


Random forest regressor

In [50]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import matplotlib.pyplot as plt

# Features and target
X = sol.drop(columns=["next_price_var"])
y = sol["next_price_var"]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R-squared (R²): {r2:.4f}")

In [ ]:
from catboost import CatBoostRegressor
import numpy as np
# Train Catboost
model = CatBoostRegressor(verbose = 0, random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
# Sign accuracy: correct direction (up/down)
sign_accuracy = (np.sign(y_pred) == np.sign(y_test)).mean()

print(f"Sign Accuracy: {sign_accuracy:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R-squared (R²): {r2:.4f}")

In [ ]:
# Feature importances
importances = model.feature_importances_
feat_importances = pd.Series(importances, index=X.columns)

# Sort and plot
feat_importances.sort_values(ascending=True).plot(kind='barh', figsize=(8,6))
plt.title('Feature Importances')
plt.xlabel('Importance')
plt.show()

## Random forest classifier

In [ ]:
sigma = sol["next_price_var"].std()

sol["target"] = sol["next_price_var"].apply(
                    lambda x: -1 if x < -sigma else (1 if x > sigma else 0)
                    )
X = sol.drop(columns=["next_price_var", "target"])
y = sol["target"]

X = X.dropna()
y = y.loc[X.index]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest Classifier
# clf = RandomForestClassifier(n_estimators=100, random_state=42)
# Train Catboost Classifier
clf = CatBoostClassifier(verbose=0,random_state=42)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# # Feature Importances
# importances = clf.feature_importances_
# feat_importances = pd.Series(importances, index=X.columns)

# feat_importances.sort_values(ascending=True).plot(kind='barh', figsize=(8,6))
# plt.title('Feature Importances')
# plt.xlabel('Importance')
# plt.show()
